# 在Agent中添加RAG

## 引入依赖

In [3]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.llms.openai_like import OpenAILike

## 在系统设置中添加LLM
我们之前直接传递了LLM，但现在我们需要在多个地方使用它，所以我们将在全局设置中添加它。

In [9]:
Settings.llm = OpenAILike(
    model="deepseek-chat",
    api_base='https://api.deepseek.com',
    api_key='sk-2142926dcf7f4d12aa0044e638c33bc9',
    is_chat_model=True,
    timeout=10*60,
)

嵌入自定义的embed模型

In [11]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="/Users/yangzy/Downloads/bge-base-zh-v1.5"
)

## 加载和索引文档
我们现在将快速连续地做三件事情：我们将从名为“data”的文件夹中加载PDF，使用VectorStoreIndex对其进行索引和嵌入，然后从该索引创建一个查询引擎。

In [12]:
documents = SimpleDirectoryReader("../data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


我们可以快速运行一个冒烟测试，以确保引擎正常工作

In [20]:
response = query_engine.query(
    "2024年国内增值税预算收入多少?"
)
print(response)

2024年国内增值税预算收入为36085.00亿元。


## 添加一个查询引擎tool

In [14]:
from llama_index.core.tools import QueryEngineTool

现在我们将查询引擎转变为工具，通过提供适当的元数据

In [29]:
budget_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="chinese_budget_2024",
    description="查询2024年国内预算的tool",
)

我们通过在我们的工具数组中添加这个引擎来修改我们的代理（我们还删除了llm参数，因为它现在由设置提供）

In [30]:
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool

def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

agent = ReActAgent.from_tools([multiply_tool, add_tool, budget_tool], verbose=True)

## 使用多个tools提问问题

In [27]:
response = agent.chat("20+(2*4)等于多少? 使用tool去计算每一步的结果.")
print(response)

Thought: The current language of the user is: Chinese. I need to use the multiply tool to calculate the result of 2*4 first, and then use the add tool to add the result to 20.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
Thought: I have the result of 2*4, which is 8. Now I need to use the add tool to add this result to 20.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: 28
28


In [34]:
response = agent.chat("2024年国内增值税预算收入金额乘以3是多少？")
print(response)

Thought: (Implicit) I can answer without any more tools!
Answer: 我是DeepSeek Chat，一个由中国深度求索公司开发的智能助手。
我是DeepSeek Chat，一个由中国深度求索公司开发的智能助手。
